In [1]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pickle

from skimage import color
from skimage import io
from keras.utils import plot_model
from keras import regularizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers import BatchNormalization
from keras import initializers
from keras.datasets import mnist

In [2]:
%load_ext autotime
RANDOM_STATE = 42

In [3]:
MEMORY_LIMIT = 4800
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

time: 99.1 ms


In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

time: 54.8 ms


In [5]:
tf.test.is_gpu_available()

False

time: 78.3 ms


In [6]:
tf.test.is_built_with_cuda()

True

time: 72.5 ms


In [7]:
mnist = tf.keras.datasets.mnist

time: 69 ms


In [8]:
#from keras.datasets import cifar10
# load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

time: 308 ms


In [9]:
X_train, X_test = X_train / 255.0, X_test / 255.0

time: 144 ms


In [10]:
learning_rates = [0.1,0.01,0.001,0.0001,0.00001]
batch_sizes = [32,48,64,96,128]
opt_layer_1=0.001
opt_layer_2=32
temp=0
test_acc=0
combinations=0
flag=0

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        
        print(learning_rate,batch_size)

        model = Sequential()

        model.add(Conv2D(32, (5, 5),strides = [1,1], input_shape=(28, 28, 1),padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(32, (3, 3),padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Flatten())

        model.add(Dense(10))
        model.add(Activation('softmax'))

        my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]

        opt = SGD(lr = learning_rate)
        model.compile(
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      optimizer=opt,
                      metrics=['accuracy'],
                      )

        model.fit(X_train,y_train,
                  batch_size= batch_size,
                  epochs=10,
                  validation_data=(X_test, y_test),
                  callbacks=[my_callbacks])
        
        _,test_acc = model.evaluate(X_test, y_test)

        print("Current Acc: ",test_acc)
        print("Best Acc: ",temp)
        combinations=combinations+1

        if flag==0 :
            if test_acc > temp :
                opt_layer_1 = learning_rate
                temp=test_acc
            learning_rates = [opt_layer_1]

        elif flag==1 :
            if test_acc > temp :
                opt_layer_2 = batch_size
                temp=test_acc 
            batch_sizes = [opt_layer_2]
    flag=1
    test_acc=0

##################################################################################
print("Trials : ",combinations)
print("Optimal Validation Accuracy: ",temp)
print("The Final Optimal Values for all block is : ")
print('learning_rates: ',opt_layer_1,'\n batch_sizes: ',opt_layer_2)

0.1 32
Epoch 1/10
1875/1875 [==============================] - 26s 13ms/step - loss: 0.3987 - accuracy: 0.8722 - val_loss: 0.0851 - val_accuracy: 0.9729
Epoch 2/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0590 - accuracy: 0.9822 - val_loss: 0.0503 - val_accuracy: 0.9835
Epoch 3/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0408 - accuracy: 0.9873 - val_loss: 0.0376 - val_accuracy: 0.9882
Epoch 4/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0363 - accuracy: 0.9884 - val_loss: 0.0406 - val_accuracy: 0.9852
Epoch 5/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0305 - accuracy: 0.9904 - val_loss: 0.0319 - val_accuracy: 0.9888
Epoch 6/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0260 - accuracy: 0.9918 - val_loss: 0.0354 - val_accuracy: 0.9887
Epoch 7/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0243 - accuracy: 0.9929 - val_loss: 0

1250/1250 [==============================] - 22s 18ms/step - loss: 0.0524 - accuracy: 0.9842 - val_loss: 0.0369 - val_accuracy: 0.9888
Epoch 4/10
1250/1250 [==============================] - 21s 17ms/step - loss: 0.0411 - accuracy: 0.9870 - val_loss: 0.0343 - val_accuracy: 0.9890
Epoch 5/10
1250/1250 [==============================] - 24s 19ms/step - loss: 0.0336 - accuracy: 0.9899 - val_loss: 0.0366 - val_accuracy: 0.9881
Epoch 6/10
1250/1250 [==============================] - 24s 19ms/step - loss: 0.0301 - accuracy: 0.9908 - val_loss: 0.0305 - val_accuracy: 0.9902
Epoch 7/10
1250/1250 [==============================] - 23s 18ms/step - loss: 0.0283 - accuracy: 0.9911 - val_loss: 0.0335 - val_accuracy: 0.9888
Epoch 8/10
1250/1250 [==============================] - 23s 18ms/step - loss: 0.0231 - accuracy: 0.9921 - val_loss: 0.0301 - val_accuracy: 0.9897
Epoch 9/10
1250/1250 [==============================] - 22s 18ms/step - loss: 0.0208 - accuracy: 0.9942 - val_loss: 0.0307 - val_accura